In [ ]:
#토큰 불러오기
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("sarcasm_0526")

In [3]:
#llm 모델 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.2, model_name="gpt-4o-mini")

In [ ]:
# prompt 가져오기
from langchain_core.prompts import load_prompt

prompt = load_prompt(
    "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/generate_context.yaml",
    encoding="utf-8",
)
prompt

In [5]:
# ① 출력 데이터 모델 정의 (score, reason)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional

class SentimentResult(BaseModel):
    context: str = Field(description="대화 상황을 한 문장으로 요약")
    sentiment: str = Field(description="상황의 긍정/부정 분류")

output_parser = PydanticOutputParser(pydantic_object=SentimentResult)

In [6]:
format_instructions = output_parser.get_format_instructions()
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
# 체인 생성
chain = prompt | llm | output_parser

In [ ]:
# 데이터셋 1000개 만들기
import os
import shutil

source_folder = "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/TL_05. NATEON"
target_folder = "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/sixth_data_0526"
os.makedirs(target_folder, exist_ok=True)

# JSON 파일 목록
json_files = [f for f in os.listdir(source_folder) if f.endswith(".json")]

# 앞 1000개만 복사
for file in json_files[500:600]:
    shutil.copy(os.path.join(source_folder, file), os.path.join(target_folder, file))

print("앞 100개 JSON 파일을 복사했습니다.")

In [ ]:
# 파일 안에 json 파일 개수 파악
import os

folder_path = "/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/sixth_data_0526"

json_count = sum(
    1 for file in os.listdir(folder_path)
    if file.endswith(".json")
)

print(f"JSON 파일 개수: {json_count}개")

In [34]:
#json 파일 불러오기
import os
import json

entries = []

# 폴더 안의 모든 JSON 파일 처리
json_folder_path = '/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/Dataset/sixth_data_0526'

for filename in os.listdir(json_folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(json_folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f) 

        # info 추출
        for item in data['info']:
            annotations = item['annotations']
            subject = annotations.get('subject', '')
            speaker_type = annotations.get('speaker_type', '')
            text = annotations.get('text', '')

            # dict 형태로 저장
            entry = {
                "subject": subject,
                "speaker_type": speaker_type,
                "text": text
            }
            entries.append(entry)

In [35]:
# 결과를 저장할 리스트
results = []

for entry in entries:
    # 체인 호출
    parsed = chain.invoke({
        "subject": entry["subject"],
        "speaker_type": entry["speaker_type"],
        "text": entry["text"]
    })
    
    # 결과 저장
    results.append({
        "subject": entry["subject"],
        "speaker_type": entry["speaker_type"],
        "context": parsed.context,
        "sentiment": parsed.sentiment
    })

In [ ]:
test_df_1 = test_df[test_df["sentiment"] == '부정']
test_df_1.head()

In [38]:
test_df_1.to_csv("/Users/seojeonghun/Documents/langchain-kr/20-Projects/sarcasm/df/negative_data_6_0531.csv", encoding = "utf-8-sig")

In [ ]:
test_df["sentiment"].value_counts()